In [1]:
import os
import json 

from oddstracker.config import DATA_DIR
from oddstracker.domain.kambi_event import KambiData, KambiEvent
from oddstracker.domain.kambi_provider import PROVIDER, KambiProvider
from oddstracker.service.oddscollector import collect_kdata, store_kdata

SQL tables already exist: ['betoffer', 'event']


In [3]:
kdata = collect_kdata()

In [8]:
with open(os.path.join(DATA_DIR, "kdata.json"), "w", encoding="utf-8") as f:
    _test_data = kdata[0]
    f.write(_test_data.model_dump_json())

In [ ]:
with open(os.path.join(DATA_DIR, "betOffers.json"), "w") as f:
    json.dump(kdata[0].betOffers[0].model_dump(), f, indent=2, default=str)
